In [ ]:
import numpy as np# linear algebra
import pandas as pd 
import seaborn as sns
import sklearn
import warnings
import io
import requests
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set()
import xgboost as xgb
from sklearn.metrics import r2_score

from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Load and check data

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv')
sample = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv')

Below is the first 5 rows of test dataset:

In [ ]:
train.head()

The dimension and number of missing values in the train dataset is as below:

In [ ]:
print(f'Number of rows: {train.shape[0]};  Number of columns: {train.shape[1]}; No of missing values: {sum(train.isna().sum())}')

In [ ]:
print(f'Number of rows: {test.shape[0]};  Number of columns: {test.shape[1]}; No of missing values: {sum(test.isna().sum())}')

In [ ]:
test.head()

Infos

In [ ]:
train.info()

Summarie and statistics¶

In [ ]:
train.describe().T

let create day , months and years variables

In [ ]:
train['year'] = pd.DatetimeIndex(train['date']).year
train['month'] = pd.DatetimeIndex(train['date']).month
train['day']= pd.DatetimeIndex(train['date']).day

In [ ]:
print(f'train years  {train.year.unique()}  months : {train.month.unique()}    days: {train.day.unique()} ')

In [ ]:
test['year'] = pd.DatetimeIndex(test['date']).year
test['month'] = pd.DatetimeIndex(test['date']).month
test['day']= pd.DatetimeIndex(test['date']).day

In [ ]:
print(f'test years  {test.year.unique()}  months : {test.month.unique()}    days: {test.day.unique()} ')

In [ ]:
sns.countplot(x='product', data=train, palette='tab10')

In [ ]:
sns.countplot(x='country', data=train, palette='tab10')

In [ ]:
sns.countplot(x='store', data=train, palette='tab10')

In [ ]:
#histogram
sns.histplot(train['num_sold'],color="red", label="100% Equities", kde=True, stat="density", linewidth=0)

In [ ]:
print ("Skew is:", train.num_sold.skew())
plt.hist(train.num_sold, color='red')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train['date'], train['num_sold'], label='num_sold') #The parameter label is to indicate the legend. This doesn't mean that it will be shown, we'll have to use another command that I'll explain later.

In [ ]:
target = np.log(train.num_sold)
print ("Skew is:", target.skew())
plt.hist(target, color='blue')
plt.show()

In [ ]:
plt.plot(train['date'], train['num_sold'], label='num_sold')
plt.xlabel('Date')
plt.ylabel('Popularity')
plt.title('Popularity of AI terms by date')
plt.grid(True)
plt.legend()

In [ ]:
num_sold_by_country = train.groupby(['country'])[['num_sold']].mean()
num_sold_by_country 

In [ ]:
colors_4 = ['magenta','yellow','orange','red']
colors_3 = ['green', 'blue','cyan']

In [ ]:
num_sold_by_country.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_country.index,autopct='%1.1f%%', colors=colors_4)
plt.show()

In [ ]:
num_sold_by_store = train.groupby(['store'])[['num_sold']].mean()
num_sold_by_store.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_store.index,autopct='%1.1f%%', colors=colors_4)
plt.show()


In [ ]:
num_sold_by_product = train.groupby(['product'])[['num_sold']].mean()
num_sold_by_product.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_product.index,autopct='%1.1f%%', colors=colors_4)
plt.show()

In [ ]:
num_sold_by_year= train.groupby(['year'])[['num_sold']].mean()
num_sold_by_year.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_year.index,autopct='%1.1f%%', colors=colors_4)
plt.show()

In [ ]:
num_sold_by_month= train.groupby(['month'])[['num_sold']].mean()
num_sold_by_month.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_month.index,autopct='%1.1f%%', colors=colors_4)
plt.show()

In [ ]:
num_sold_by_day= train.groupby(['day'])[['num_sold']].mean()
num_sold_by_day.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_day.index,autopct='%1.1f%%', colors=colors_4)
plt.show()

function select_date is function give you data between two dates 

In [ ]:
def select_date(date1,date2,train):

 
 df = train[(train['date'] >= date1) & (train['date'] <= date2)]
 return df

data train between 2015-01-01 and 2015-12-30

In [ ]:
date1='2015-01-01'
date2='2015-12-30'
df=select_date(date1,date2,train)

In [ ]:
df.head()

In [ ]:
def plt_sns(train):
 sns.countplot(x='product', data=train, palette='tab10')
 sns.countplot(x='country', data=train, palette='tab10')
 sns.countplot(x='store', data=train, palette='tab10')


def plt_his(train):
  sns.histplot(train['num_sold'],color="red", label="100% Equities", kde=True, stat="density", linewidth=0)
  print ("Skew is:", train.num_sold.skew())
  plt.hist(train.num_sold, color='red')
  plt.show()
  plt.plot(train['date'], train['num_sold'], label='num_sold') #The parameter label is to indicate the legend. This doesn't mean that it will be shown, we'll have to use another command that I'll explain later.
  target = np.log(train.num_sold)
  print ("Skew is:", target.skew())
  plt.hist(target, color='blue')
  plt.show()
  plt.plot(train['date'], train['num_sold'], label='num_sold')
  plt.xlabel('Date')
  plt.ylabel('Popularity')
  plt.title('Popularity of AI terms by date')
  plt.grid(True)
  plt.legend()



colors_4 = ['magenta','yellow','orange','red']
colors_3 = ['green', 'blue','cyan']

def num_sold_by_country_pie(train):
  num_sold_by_country = train.groupby(['country'])[['num_sold']].mean()
  num_sold_by_country.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_country.index,autopct='%1.1f%%', colors=colors_4)
  plt.show()

def num_sold_by_store_pie(train):
    num_sold_by_store = train.groupby(['store'])[['num_sold']].mean()
    num_sold_by_store.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_store.index,autopct='%1.1f%%', colors=colors_4)
    plt.show()
    
def num_sold_by_product_pie(train):
  num_sold_by_product = train.groupby(['product'])[['num_sold']].mean()
  num_sold_by_product.plot.pie(subplots=True, figsize=(20,10), labels=num_sold_by_product.index,autopct='%1.1f%%', colors=colors_4)
  plt.show()

In [ ]:
num_sold_by_product_pie(df)

In [ ]:
plt_his(df)

In [ ]:
num_sold_by_country_pie(df)

In [ ]:
num_sold_by_store_pie(df)

In [ ]:
num_sold_by_product_pie(df)

In [ ]:
y     = train[['row_id','num_sold']]

train.drop(['date','num_sold'] , axis = 1 , inplace = True)
test.drop('date' , axis = 1 , inplace = True)


In [ ]:
labelencoder=LabelEncoder()
all_df = [train, test]
all_df = pd.concat(all_df).reset_index(drop=True)
all_df['country']      = labelencoder.fit_transform(all_df['country'])
all_df['store']   = labelencoder.fit_transform(all_df['store'].astype(str))
all_df['product']   = labelencoder.fit_transform(all_df['product'].astype(str))

In [ ]:
X = pd.DataFrame(all_df[:26298])
test  = pd.DataFrame(all_df[26298:32868])

In [ ]:
sns.heatmap(X.corr(), annot=True, fmt='.2f')

In [ ]:
sns.pairplot(X)

In [ ]:
sns.jointplot(x='store', y='country', data=X)

In [ ]:
# explore lightgbm number of trees effect on performance
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y['num_sold'], test_size=0.2, random_state=42)


In [ ]:
from xgboost import XGBRegressor
XGB = XGBRegressor(max_depth=2,learning_rate=0.1,n_estimators=1000,reg_alpha=0.001,reg_lambda=0.000001,n_jobs=-1,min_child_weight=3)
XGB.fit(X_train,y_train)

In [ ]:
from lightgbm import LGBMRegressor
LGBM = LGBMRegressor(n_estimators = 1000)
LGBM.fit(X_train,y_train)

In [ ]:
print ("Training score:",XGB.score(X_train,y_train), "Test Score:",XGB.score(X_test,y_test))
print ("Training score:",LGBM.score(X_train,y_train),"Test Score:",LGBM.score(X_test,y_test))

In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
clf = DecisionTreeClassifier()
scoring = 'accuracy'
score = cross_val_score(clf, X_train,y_train, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# decision tree Score
round(np.mean(score)*100, 2)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
# calculate manually
def my_function(y,y_preds):
  from sklearn.metrics import r2_score
  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import mean_absolute_error  

  m1=mean_absolute_error(y,y_preds)
  m2 = mean_squared_error(y,y_preds)
  r2 = r2_score(y, y_preds)

  print("Results by manual calculation:")
  print("mean_absolute_error:",m1)
  print("mean_squared_error:", m2)
  print('r2 score for a worse model is', r2)

In [ ]:
y_preds_lgbm = LGBM.predict(test)
my_function(sample.num_sold,y_preds_lgbm)

In [ ]:
y_XGB = XGB.predict(test )
my_function(sample.num_sold,y_XGB)

In [ ]:
nn_preds = clf.predict(test )
my_function(sample.num_sold,nn_preds)

In [ ]:
test.head()

In [ ]:
my_submission = pd.DataFrame({'row_id': test.row_id, 'num_sold': y_preds_lgbm})
# you could use any filename. We choose submission here
my_submission.to_csv('submission_ran.csv', index=False)
my_submission.head()